In [1]:
import numpy as np
from copy import deepcopy

# Part 1

In [2]:
test1 = '8A004A801A8002F478'
test2 = '620080001611562C8802118E34'
test3 = 'C0015000016115A2E0802F182340'
test4 = 'A0016C880162017C3686B18A3D4780'

In [3]:
def hex_to_binary(hexi):
    table = {'0':'0000', '1':'0001', '2':'0010', '3':'0011', '4':'0100',
             '5':'0101', '6':'0110', '7':'0111', '8':'1000', '9':'1001',
             'A':'1010', 'B':'1011', 'C':'1100', 'D':'1101', 'E':'1110',
             'F':'1111'}
    
    bini = ''
    for c in hexi:
        bini += table[c]
        
    return bini

def header(bini, depth):
    vers, bini = bini[:3], bini[3:]
    v = int(vers, 2)
    
    tpid, bini = bini[:3], bini[3:]
    t = int(tpid, 2)
    
    if t == 4: #literal value
        bini, val = literal_value(bini)
        
    else: #operator
        bini, val, ver = operator(bini, t, deepcopy(depth))
        v += ver
        
    return v, t, val, bini
            
def literal_value(bini):
    op_bin = ''
    value = ''
    while bini[0] == '1':
        value += bini[1:5]
        op_bin += bini[:5]
        bini = bini[5:]

    #final block starts with 0
    value += bini[1:5]
    op_bin += bini[:5]
    bini = bini[5:]
    
    val =  int(value, 2)
    return bini, val

def operator(bini, t, depth):
    op_bin = ''
    
    depth += 1
    i, bini = bini[0], bini[1:]
    
    op_bin += i
    
    if i == '0':
        size = 15
    elif i == '1':
        size = 11
    
    length, bini = bini[:size], bini[size:]
    lng = int(length, 2)
    
    op_bin += ' '+length
    
    ver = 0
    vals = []
    if i == '0':
        sub_bini, bini = bini[:lng], bini[lng:]
        op_bin += ' '+sub_bini
        while len(sub_bini):
            v, _, val, sub_bini = header(sub_bini, deepcopy(depth))
            vals.append(val)
            ver += v
            if len(sub_bini) < 6:
                break
        
    elif i == '1':
        for j in range(0, lng):
            tbini = deepcopy(bini)
            
            v, _, val, bini = header(bini, deepcopy(depth))
            vals.append(val)
            ver += v
            
            if len(bini) > 0:
                tbini = tbini[:-len(bini)]
            op_bin += ' '+tbini
                       
    if t == 0:
        op_val = np.sum(vals, dtype=np.int64)
    elif t == 1:
        op_val = np.prod(vals, dtype=np.int64)
    elif t == 2:
        op_val = np.min(vals)
    elif t == 3:
        op_val = np.max(vals)
    elif t == 5:
        op_val = int(vals[0] > vals[1])
    elif t == 6:
        op_val = int(vals[0] < vals[1])
    elif t == 7:
        op_val = int(vals[0] == vals[1])
    else:
        print(t)
        
    return bini, op_val, ver

def parse_packet(data):
    bini = hex_to_binary(data)
    version = 0
    
    depth = 0
    
    while len(bini) > 6:
        if int(bini, 2) == 0:
            break
        vers, tpid, val, bini = header(bini, depth)
        version += vers
    return version, val

#print(parse_packet('D2FE28'))
#print(parse_packet('38006F45291200'))
print(parse_packet(test1)[0])
print(parse_packet(test2)[0])
print(parse_packet(test3)[0])
print(parse_packet(test4)[0])

16
12
23
31


In [4]:
inpt = np.genfromtxt('day16_input.txt', dtype=str, delimiter='\n')
inpt = str(inpt)
print('Part1 Result:', parse_packet(inpt)[0])

Part1 Result: 943


# Part 2

In [5]:
tests = ['C200B40A82',
         '04005AC33890',
         '880086C3E88112',
         'CE00C43D881120',
         'D8005AC2A8F0',
         'F600BC2D8F',
         '9C005AC2F8F0',
         '9C0141080250320F1802104A08']

for test in tests:
    print(parse_packet(test)[1])

3
54
7
9
1
0
0
1


In [6]:
print('Part2 Result:', parse_packet(inpt)[1])

Part2 Result: 167737115857
